<a href="https://colab.research.google.com/github/ZeroTimo/Finetuning_Mistral7b/blob/main/FrankensteinQLoRAStarter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
# !pip install -q -U pandas # you don't need to install either of these last two libs if you're using Colab
# !pip install -q -U torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
!pip install huggingface-cli

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli


In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `learn` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [22]:
import random
import torch
import pandas as pd
from datasets import Dataset
import peft
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)

set_seed()

In [23]:
mistral7b = 'mistralai/Mistral-7B-v0.1'
# STEP 1. Check and make sure you're using the right model and notebook here.
model_name = mistral7b

## EDA

In [24]:
df = pd.read_csv("frankenstein_chunks.csv")
df.head()

,text
0,﻿The Project Gutenberg eBook of Frankenstein; ...
1,Further corrections by Menno de Leeuw.\n\n\n**...
2,"I am already far north of London, and as I wal..."
3,Its productions and features may be without ex...
4,But supposing all these conjectures to be fals...


In [25]:
print("Dataframe Info:")
print(df.info())
print("\n")
print("Dataframe Description:")
print(df.describe())
print("\n")
print("Number of unique values in each column:")
print(df.nunique())
random_index= random.randint(0, len(df) - 1)
df.loc[random_index, 'text']

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    481 non-null    object
dtypes: object(1)
memory usage: 3.9+ KB
None


Dataframe Description:
                                                     text
count                                                 481
unique                                                481
top     International donations are gratefully accepte...
freq                                                    1


Number of unique values in each column:
text    481
dtype: int64


'The thatch had fallen in, the walls were unplastered, and the\ndoor was off its hinges. I ordered it to be repaired, bought some\nfurniture, and took possession, an incident which would doubtless have\noccasioned some surprise had not all the senses of the cottagers been\nbenumbed by want and squalid poverty. As it was, I lived ungazed at\nand unmolested, hardly thanked for the pittance of food and clothes\nwhich I gave, so much does suffering blunt even the coarsest sensations\nof men.\n\nIn this retreat I devoted the morning to labour; but in the evening,\nwhen the weather permitted, I walked on the stony beach of the sea to\nlisten to the waves as they roared and dashed at my feet. It was a\nmonotonous yet ever-changing scene. I thought of Switzerland; it was\nfar different from this desolate and appalling landscape. '

In [6]:
df.isnull().sum()

,0
text,0


In [27]:
# Now we'll quickly convert this to a train/test split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)

# STEP 2. Convert the train_df and test_df from Pandas into Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


## Model Import and Tokenization

In [28]:
quant_config = BitsAndBytesConfig(
  # STEP 3. Pass the appropriate parameters here to 4-bit quantize the model, then instantiate the model and check what it's running on.
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",  # Let transformers choose the best device
    trust_remote_code=True # Mistral requires this
)
print("\n\nModel is running on:" + "\n")
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]



Model is running on:



MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [14]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# STEP 4. Prepare the model for QLoRA. Configure LoRA for our finetuning run. Then tokenize the data.
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.005,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_train_dataset= train_dataset.map(lambda examples: tokenizer(examples['text']), batched=True)
tokenized_test_dataset = test_dataset.map(lambda examples: tokenizer(examples['text']), batched=True)

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

## Base Model Evaluation

In [29]:
def generate_text(prompt):
  device = "cuda"
  inputs = tokenizer(prompt, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, max_new_tokens=100)
  output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return output

In [16]:
# STEP 5. Generate a completion with the base model for informal evaluation.
base_generation = generate_text("I'm afraid I've created a ")
base_generation

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"I'm afraid I've created a 2000-level problem with a 100-level solution.\n\nI'm a 2000-level problem.\n\nI'm a 2000-level problem.\n\nI'm a 2000-level problem.\n\nI'm a 2000-level problem.\n\nI'm a 2000-level problem.\n\nI'm a 2"

In [30]:
def calc_perplexity(model):
  total_perplexity = 0
  device = "cuda"  # Define device here
  model.to(device)  # Move the model to the GPU
  for row in test_dataset:
    # Move inputs to the GPU
    inputs = tokenizer(row['text'], return_tensors="pt").to(device)
    input_ids = inputs["input_ids"] # input_ids is already on the GPU due to the line above
    # Calculate the loss without updating the model
    with torch.no_grad():
        outputs = model(**inputs, labels=input_ids)
    loss = outputs.loss
    # Calculate perplexity
    perplexity = torch.exp(loss)
    total_perplexity += perplexity

  num_test_rows = len(test_dataset)
  avg_perplexity = total_perplexity / num_test_rows
  return avg_perplexity

base_ppl = calc_perplexity(model)
base_ppl

tensor(8.6040, device='cuda:0')

## Training

Make sure you can leave your browser open for a while. This took about 15 minutes on a Colab T4 GPU.

In [40]:
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

config = LoraConfig(
    r=8,              # LoRA rank
    lora_alpha=16,     # LoRA alpha
    target_modules=["q_proj", "v_proj"], # Target modules for Mistral
    lora_dropout=0.05,  # Dropout
    bias="none",       # Bias type
    task_type="CAUSAL_LM" # Task type
)

model = get_peft_model(model, config)
model.print_trainable_parameters() # Print trainable parameters

tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        warmup_steps=2,
        fp16=True,
        logging_steps=1,
        save_steps=200,
        output_dir="outputs",
      # STEP 7. Configure the training arguments.
        per_device_train_batch_size=2,
        num_train_epochs=2,
        learning_rate=2e-5,
        optim="paged_adamw_8bit",
        report_to="none"  # Disable all reporting (including wandb)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
# STEP 8. Finetune the model.
trainer.train()

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


Step,Training Loss
1,2.415400
2,2.051900
3,1.227500
4,2.237200
5,1.288800
6,2.110500
7,2.229900
8,1.372500
9,1.206600
10,2.098000


TrainOutput(global_step=384, training_loss=1.8897206919888656, metrics={'train_runtime': 648.8879, 'train_samples_per_second': 1.184, 'train_steps_per_second': 0.592, 'total_flos': 8197199044263936.0, 'train_loss': 1.8897206919888656, 'epoch': 2.0})

## Evaluating the finetuned model

In [41]:
# STEP 9. Generate a completion with the finetuned model and compare it to the base generation.
ft_generation = generate_text("I'm afraid I've created a ")

print("Base model generation: " + base_generation + "\n\n")
print("Finetuned generation: " + ft_generation)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Base model generation: I'm afraid I've created a 2000-level problem with a 100-level solution.

I'm a 2000-level problem.

I'm a 2000-level problem.

I'm a 2000-level problem.

I'm a 2000-level problem.

I'm a 2000-level problem.

I'm a 2


Finetuned generation: I'm afraid I've created a  monster.

I've been working on a new project for the past few months. It's a new way to use the 3D printer to make a 3D printer.

I've been working on a new project for the past few months. It's a new way to use the 3D printer to make a 3D printer.

I've been working on a new project for the past few months. It's a new


A little more like the original text, right? Try experimenting with the hyperparameters to see if you can improve performance.

In [42]:
# STEP 10. Calculate the finetuned model's perplexity and compare it to the base model's.
ft_ppl = calc_perplexity(model)
print("Base model perplexity: " + str(base_ppl))
print("Finetuned model perplexity: " + str(ft_ppl))

Base model perplexity: tensor(8.6040, device='cuda:0')
Finetuned model perplexity: tensor(6.6656, device='cuda:0')
